# ![PUBG](https://i.imgur.com/0hB3OfZ.jpg)

# Содержание
- [Data Description](#Data Description)
- [Exploratory Data Analysis](#Exploratory Data Analysis)
- [Feature Engineering](#Конструирование признаков(Feature Engineering)
- [Solution](#Solution)

# Data Description

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
np.random.seed(7)

train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv') #Считываем файл CSV
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv') #Считываем файл CSV
print(train.shape)
print(test.shape)

### Определение данных из датасета

* groupId - ID для идентификации группы в матче. Если одна и та же группа игроков играет в разных матчах, у них будет разный groupId каждый раз.
* matchId - ID для идентификации матча.
* assists - Количество вражеских игроков, поврежденных этим игроком и убитых товарищами по команде.
* boosts - Количество использованных бустеров.
* damageDealt - Общий урон. Примечание: урон по самому себе вычитается.
* DBNOs - Количество игроков, которые были кнокнуты.
* headshotKills - Количество убитых игроков выстрелами в голову.
* heals - Количество использованных лечащих предметов.
* killPlace - Место, которое игрок занял в матче.
* killPoints - Очки, основанные на рейтинге игроков. (Думайте об этом как о рейтинге Эло, где важны только убийства).
* kills - Количество убийств.
* killStreaks - Максимальное количество вражеских игроков, убитых за короткое время.
* longestKill - Наибольшее расстояние между игроком и игроком, убитым в момент смерти. Это может вводить в заблуждение, так как отстранение игрока от игры и отъезд могут привести к большой статистике.
* maxPlace - Худшее место у нас есть данные в матче. Это может не совпадать с numGroups, так как иногда данные пропускаются по местам размещения.
* numGroups - Количество групп, по которым у нас есть данные в матче.
* revives - Сколько раз этот игрок восстанавливал товарищей по команде.
* rideDistance - Общее пройденное расстояние в транспортных средствах, измеренное в метрах.
* roadKills - Количество убийств в автомобиле.
* swimDistance - Общее расстояние, пройденное плаванием в метрах.
* teamKills - Сколько раз этот игрок убивал товарища по команде.
* vehicleDestroys - Количество уничтоженных автомобилей.
* walkDistance - Общее пройденное расстояние пешком в метрах.
* weaponsAcquired - Количество поднятого оружия.
* winPoints - Победный внешний рейтинг игрока. (Думайте об этом как о рейтинге Эло, где важен только выигрыш).
* winPlacePerc - Цель прогнозирования. Это выигрышное размещение в процентилях, где 1 соответствует 1-му месту, а 0 соответствует последнему месту в матче. Он рассчитывается по maxPlace, а не по numGroups, поэтому в совпадении могут быть пропущены фрагменты.

# Exploratory Data Analysis
![](https://i.gifer.com/Cwgf.gif)

#### Ищем игры в которых winPlacePerc является NaN

In [ ]:
count = train.shape # Cоздадим переменную, чтобы в итоге узнать сколько было удалено строк.
train[train['winPlacePerc'].isnull()]

Удаляем эту строку, потому что в них был только один игрок

In [ ]:
train.drop(2744604, inplace = True)

## Очистка датасета от читеров
# ![](https://media.giphy.com/media/PXot7Kx1tUVos/giphy.gif)

#### Ищем игроков, которые совершали убийства без пройденного расстояния(без движений)

In [ ]:
train['totalDistance'] = train['rideDistance'] + train['walkDistance'] + train['swimDistance']
train['killWithoutMoving'] = ((train['kills'] > 2) & (train['totalDistance'] == 0))

#### Также добавляем процент убийств в голову, что также поможет выявить читеров

In [ ]:
train['hs_ratio'] = train['headshotKills'] /  train['kills']
train['hs_ratio'] = train['hs_ratio'].fillna(0)

Выводим наших читеров и удаляем их из нашей модели

In [ ]:
train[train['killWithoutMoving'] == True]

In [ ]:
train.drop(train[train['killWithoutMoving'] == True].index, inplace = True)

Удалим этот столбцы, которые нам больше не понадобятся

In [ ]:
train = train.drop('killWithoutMoving', 1) 
train = train.drop('totalDistance', 1)

#### Теперь посмотрим на тех, кто совершил аномальное количество убийств на транспорте и удалим их из нашей модели
# ![](https://i.gifer.com/7Jj0.gif)

In [ ]:
train[train['roadKills'] > 10]

In [ ]:
train.drop(train[train['roadKills'] > 10].index, inplace = True)

### AIM (точность стрельбы) игроков
# ![](https://media.giphy.com/media/26xiwWAW1d6UEZfpu/giphy.gif)

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['hs_ratio'], bins=10)
plt.show()

#### Рассмотрим расстояния с которых были сделаны убийства

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['longestKill'], bins = 10)
plt.show()

Удалим тех, кто совершил убийства с расстояния больше 600 метров

In [ ]:
train.drop(train[train['longestKill'] > 600].index, inplace = True)

#### Посмотрим кто имеет 10 и больше убийств с 100% попаданий в голову и уберем их

In [ ]:
train[(train['hs_ratio'] == 1) & (train['kills'] > 9) & (train['assists'] == 0 )].head(20)

In [ ]:
train.drop(train[(train['hs_ratio'] == 1) & (train['kills'] > 9) & (train['assists'] == 0 )].index, inplace = True)

#### Построим спектрограмму по количеству убийств и соответствующему ему количеству игроков

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['kills'], bins = 10)
plt.show()

Удалим из нашей модели тех, кто совершил больше 45 убийств

In [ ]:
train.drop(train[train['kills'] > 45].index, inplace = True)

### Walk Distance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['walkDistance'], bins = 10)
plt.show()

Можно спокойно удалить тех, кто прошел больше 10 000 метров

In [ ]:
train.drop(train[train['walkDistance'] > 10000].index, inplace = True)

### Swim Distance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['swimDistance'], bins = 10)
plt.show()

Можно спокойно удалить тех, кто проплыл больше 1000 метров

In [ ]:
train.drop(train[train['swimDistance'] > 1000].index, inplace = True)

### Ride Distance

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(train['rideDistance'], bins = 10)
plt.show()

Удалим тех, кто проехал больше 20000 метров

In [ ]:
train.drop(train[train['rideDistance'] > 20000].index, inplace = True)

### Итог очистки

In [ ]:
print(train.shape)

Мы удалили 2238 строк из нашего дата сета

# Feature Engineering
![](https://media.giphy.com/media/39t19N2ICoIWkPaqay/giphy.gif)

Разделим наши данные на подгруппы

In [ ]:
features = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'kills', 'killStreaks', 
            'longestKill', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired']
infos = ['matchDuration', 'matchType', 'maxPlace', 'numGroups']
ELO = ['rankPoints', 'killPoints', 'winPoints']
label = ['winPlacePerc']

Посмотрим на тепловую карту с коэффицентами корреляции Пирсона с их помощью мы понимаем на какие признаки стоит обратить внимание

In [ ]:
sample = train.sample(100000)

f,ax = plt.subplots(figsize=(15, 12))
sns.heatmap(sample[ELO + features + label].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

Как мы видим roadKills, teamKills, swimDistance и vehicleDestroys слабо влияют на наш коэффицент winPlacePerc, поэтому мы их уберем. Также rankPoints, killPoint, winPoints тоже бесполезны, но они нужны для поиска рейтинговой игры, чтобы игроки попадались с равными себе противниками, поэтому в дальнейшнем мы образуем один признак просумировав их

## Образуем несколько новых полезных для нас признаков

### Players Joined
Создаем признак, показывающий сколько игроков было в игре , этот признак поможет нам нормализовать данные

In [ ]:
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
plt.figure(figsize=(15,10))
sns.countplot(train[train['playersJoined']>=75]['playersJoined'])
plt.title('playersJoined')
plt.show()

Нормализуем kills, damageDealt, maxPlace, matchDuration

In [ ]:
train['killsNorm'] = train['kills']*((100-train['playersJoined'])/100 + 1)
train['damageDealtNorm'] = train['damageDealt']*((100-train['playersJoined'])/100 + 1)
train['maxPlaceNorm'] = train['maxPlace']*((100-train['playersJoined'])/100 + 1)
train['matchDurationNorm'] = train['matchDuration']*((100-train['playersJoined'])/100 + 1)

Нормализуем данные о подключенных игроках с помощью логарифмирования.

In [ ]:
train['log_players']=np.log10(train['playersJoined'])

### Skill
ПАБГ игра, в которой важен личный скилл игроков, поэтому создадим признак skill
![image.png](https://sun9-8.userapi.com/c200820/v200820016/314a1/07xLzjSjKMk.jpg)

In [ ]:
train['skill'] = train['headshotKills'] + 0.01 * train['longestKill'] - train['teamKills'] / (train['kills'] + 1)

In [ ]:
data = train.sample(10000)
plt.figure(figsize=(15,10))
sns.scatterplot(x='skill', y='winPlacePerc', data=data)
plt.show()

### Hs_ratio

Также при выявлении читеров, мы создали hs_ratio, который нам тоже понадобится, визуализируем зависимость

In [ ]:
data = train.sample(10000)
plt.figure(figsize=(15,10))
sns.scatterplot(x='hs_ratio', y='winPlacePerc', data=data)
plt.show()

Как мы видим, есть много людей с 0 и 1 коэффицентами, это могут быть люди, которые сделал 1 убийство, но случайно, поэтому немного изменим значения коэффицента hs_ratio

In [ ]:
def transform_hsRatio(x):
    if x == 1 or x == 0:
        return 0.5
    else: 
        return x

In [ ]:
train['hs_ratio'] = train['hs_ratio'].apply(transform_hsRatio)

In [ ]:
data = train.sample(10000)
plt.figure(figsize=(15,10))
sns.scatterplot(x='hs_ratio', y='winPlacePerc', data=data)
plt.show()

Мы получили что-то вроде Гауссовского распределения с хорошими игроками справа и плохими слева

## Distance
#### Теперь добавим признак distance, который будет вычисляться по формуле
![Дистанция](https://sun9-63.userapi.com/c854228/v854228253/1ca694/fRF7lwvbK_c.jpg)

In [ ]:
train['distance'] = (train['walkDistance'] + 0.4 * train['rideDistance'] + train['swimDistance']) * (1 / train['matchDuration'])

In [ ]:
data = train.sample(10000)
plt.figure(figsize=(15,10))
sns.scatterplot(x='distance', y='winPlacePerc', data=data)
plt.show()

## Бустеры 
Бустеры также очень важны, хорошие игроки всегда стараются быть запакованы по максимуму, потому что они увеличивают регенерацию жизней и скорость передвижения, что очень важно для победы. Поэтому добавим признак boostRatio, который вычислим по формуле
![Бустеры](https://sun9-36.userapi.com/c854228/v854228253/1ca68d/64PrTDeQiiY.jpg)

In [ ]:
train['boostRatio'] = train['boosts'] ** 2 / train['walkDistance'] ** 0.5
train['boostRatio'].fillna(0, inplace = True)
train['boostRatio'].replace(np.inf, 0, inplace=True)

In [ ]:
data = train.sample(10000)
plt.figure(figsize=(15,10))
sns.scatterplot(x='boostRatio', y='winPlacePerc', data=data)
plt.show()

## Heals 
Разберем другой важный аспект игры, это так называемые хилки(heals), они лечат персонажа, т.е. восстанавливают здоровье.Поэтому добавим еще один 
признак healsRatio, который вычислим по формуле
![Хилки](https://sun9-36.userapi.com/c854228/v854228253/1ca69b/6VoQNgWD3mw.jpg)

In [ ]:
train['healsRatio'] = train['heals'] / train['matchDuration'] ** 0.1
train['healsRatio'].fillna(0, inplace = True)
train['healsRatio'].replace(np.inf, 0, inplace=True)

In [ ]:
data = train.sample(10000)
plt.figure(figsize = (15, 10))
sns.scatterplot(x='healsRatio', y='winPlacePerc', data=data)
plt.show()

## killsRatio
#### Хорошие игроки должны делать много убийств, поэтому добавим killsRatio, а вычислим мы его по формуле
![Фраги](https://sun9-127.userapi.com/c854228/v854228253/1ca6a2/CY3zkh_Yop8.jpg)

In [ ]:
train['killsRatio'] = train['kills'] / train['matchDuration']**0.1
train['killsRatio'].fillna(0, inplace=True)
train['killsRatio'].replace(np.inf, 0, inplace=True)

In [ ]:
data = train.sample(10000)
plt.figure(figsize = (15, 10))
sns.scatterplot(x='killsRatio', y='winPlacePerc', data=data)
plt.show()

### Heatmap

#### Все мы добавили все новые признаки, теперь давайте посмотрим на новую карту корреляций

In [ ]:
engineered = ['log_players', 'matchDurationNorm', 'skill', 'hs_ratio', 'damageDealtNorm', 'distance', 'boostRatio', 'healsRatio', 'killsRatio', 'killsNorm', 'maxPlaceNorm']

In [ ]:
sample = train.sample(100000)

f,ax = plt.subplots(figsize=(15, 12))
sns.heatmap(sample[engineered + label].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

# Solution
# ![](https://www.elsetge.cat/myimg/f/80-808526_large-size-of-pubg-pubg-pc.jpg)

Приступим к самому решению для начала создадим выявленные ранее критерии для тестового датасета

In [ ]:
test['skill'] = test['headshotKills'] + 0.01 * test['longestKill'] - test['teamKills'] / (test['kills'] + 1)
def transform_hsRatio(x):
    if x == 1 or x == 0:
        return 0.5
    else: 
        return x
test['playersJoined'] = test.groupby('matchId')['matchId'].transform('count')
test['log_players'] = np.log10(test['playersJoined'])

test['killsNorm'] = test['kills']*((100-test['playersJoined'])/100 + 1)
test['damageDealtNorm'] = test['damageDealt']*((100-test['playersJoined'])/100 + 1)
test['maxPlaceNorm'] = test['maxPlace']*((100-test['playersJoined'])/100 + 1)
test['matchDurationNorm'] = test['matchDuration']*((100-test['playersJoined'])/100 + 1)

test['hs_ratio'] = test['headshotKills'] /  test['kills']
test['hs_ratio'] = test['hs_ratio'].fillna(0)        
test['hs_ratio'] = test['hs_ratio'].apply(transform_hsRatio)

test['distance'] = (test['walkDistance'] + 0.4 * test['rideDistance'] + test['swimDistance']) * (1 / test['matchDuration'])

test['boostRatio'] = test['boosts'] ** 2 / test['walkDistance'] ** 0.5
test['boostRatio'].fillna(0, inplace = True)
test['boostRatio'].replace(np.inf, 0, inplace=True)

test['healsRatio'] = test['heals'] / test['matchDuration'] ** 0.1
test['healsRatio'].fillna(0, inplace = True)
test['healsRatio'].replace(np.inf, 0, inplace=True)

test['killsRatio'] = test['kills'] / test['matchDuration']**0.1
test['killsRatio'].fillna(0, inplace=True)
test['killsRatio'].replace(np.inf, 0, inplace=True)

test.shape

Создаем y_train в которую помещаем целевую переменную, а в x_train признаки, по которым будем строить Random Forest

In [ ]:
target = 'winPlacePerc'
new = ['log_players', 'killsNorm', 'damageDealtNorm', 'maxPlaceNorm', 'matchDurationNorm']
y_train = train[target]
features = list(train.columns)
features.remove("Id")
features.remove("matchId")
features.remove("groupId")
features.remove("matchType")
features.remove("winPlacePerc")
features.remove("kills")
features.remove("damageDealt")
features.remove("maxPlace")
features.remove("matchDuration")
x_train = train[features + new]
x_test = test[features + new]
print(x_test.shape, x_train.shape, y_train.shape)

Теперь рассплитим наши данные по переменным x_train, x_val, y_train, y_val

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 1)

Создаем random forest regressor и строим его по нашим тренировочным данным.

In [ ]:
rf = RandomForestRegressor(n_estimators = 70, min_samples_leaf = 3, max_depth = 23, max_features = 0.5,
                          n_jobs = -1)

In [ ]:
rf.fit(x_train, y_train)

Проверим нашу модель по нужной нам метрике  MAE(средняя абсолютная ошибка)

In [ ]:
print('mae train: ', mean_absolute_error(rf.predict(x_train), y_train))
print('mae val: ', mean_absolute_error(rf.predict(x_val), y_val))

Присваиваем все придикты в столбец winPlacePerc в тестовом дата сете и выводим его в файл submission_rf.csv вместе с Id игроков

In [ ]:
pred = rf.predict(x_test)
test['winPlacePerc'] = pred
submission = test[['Id', 'winPlacePerc']]
submission.to_csv('submission_rf.csv', index=False)